In [1]:
from mxnet import autograd,nd

num_inputs=2
num_examples=1000
true_w=[2,-3,4]
true_b=4.2
features=nd.random.normal(scale=1,shape=(num_examples,num_inputs))
labels=true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels+=nd.random.normal(scale=0.01,shape=labels.shape)

In [11]:
from mxnet.gluon import data as gdata
batch_size=10
dataset=gdata.ArrayDataset(features,labels)  #features,labels组成一个两个元素的元组
data_iter=gdata.DataLoader(dataset,batch_size,shuffle=True)

In [12]:
for X,y in data_iter:
    print(X,y)
    break


[[ 0.02030763  1.0949801 ]
 [-1.51076    -0.9653407 ]
 [-0.3480856   0.37118176]
 [-0.7110038   1.2160763 ]
 [ 1.4106873   0.28765517]
 [ 0.12447102 -0.83142215]
 [-0.70997643 -1.008953  ]
 [-0.53713053  1.5127277 ]
 [ 0.42972964 -0.8840267 ]
 [-0.7531863   1.9670967 ]]
<NDArray 10x2 @cpu(0)> 
[ 0.9676583  4.061734   2.3854148 -0.8633205  6.1561327  6.9491215
  5.810125  -1.4106469  7.707451  -3.2198007]
<NDArray 10 @cpu(0)>


In [13]:
from mxnet.gluon import nn
net=nn.Sequential()
net.add(nn.Dense(1))

In [15]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))  #初始化时随机采样于均值为0，标准差为0.01的正态分布

In [16]:
from mxnet.gluon import loss as gloss
loss=gloss.L2Loss()

In [26]:
from mxnet import gluon
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})

In [28]:
num_epochs=3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l=loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l=loss(net(features),labels)
    print("epoch %d,loss: %f"%(epoch,l.mean().asnumpy()))

epoch 1,loss: 0.011374
epoch 2,loss: 0.000068
epoch 3,loss: 0.000048


In [30]:
dense=net[0]
true_w,dense.weight.data()

([2, -3, 4], 
 [[ 1.9999043 -3.0000894]]
 <NDArray 1x2 @cpu(0)>)

In [31]:
true_b,dense.bias.data()

(4.2, 
 [4.2003446]
 <NDArray 1 @cpu(0)>)